# Understanding the basic concepts

In [ ]:
import siibra

## Spaces, parcellations and regions

At its core, `siibra` defines different maps and reference spaces as objects. These can be accessed in the form of predefined entities, which autocomplete in most python editors, or via fuzzy string matching. Let's try a few.

In [ ]:
print("Spaces:",[s for s in siibra.spaces])
print("Parcellations:",[p for p in siibra.parcellations])

In [ ]:
space = siibra.spaces.BIG_BRAIN
space = siibra.spaces['bigbrain'] # this is equivalent

parcellation = siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_9
parcellation = siibra.parcellations['cyto'] # this is equivalent

These objects are typically linked to curated datasets in the [EBRAINS knowledge graph](https://kg.ebrainseu) (KG), using dataset identifiers.

In [ ]:
parcellation.id, space.id

In [ ]:
print(parcellation.name)
print(parcellation.description)

`siibra` also represents brain regions as semantic objects using the specific class `Region`. Since brain regions are usually part of a hierarchy, each region represents a tree which can have parent and child nodes. We can search regions from a parcellation using its `find` method. If we are confident that a region exists, we can request a unique object using `decode`, which will raise an Exception if the specification is not unique.  

In [ ]:
for region in siibra.parcellations['cyto'].find_regions('v1'):
    print("Found",region.name)

print("Last region found:",region.name)
print("Its parent:",region.parent.name)
print("The parent's parent:",region.parent.parent.name)

fp1 = siibra.parcellations['cyto'].decode_region('fp1')

## Maps

Spaces and parcellations are just semantic objects - they define a concept and provide desriptions. We are typically interested to combine them, i.e. to work with a concrete parcellation map in a particular reference space. Since EBRAINS atlases provide maps of some parcellations in multiple spaces, maps and parcellations are different concepts - maps are spatial objects, while parcellations are pure semantic objects.

In [ ]:
cytomap = siibra.parcellations['Julich 2.9'].get_map(siibra.spaces['mni152'])
print(cytomap)
print(f"This object defines {len(cytomap)} maps.")

`siibra` typically performs lazy loading of data, so that data is only retrieved when requested. As a consequence, in order to access the actual parcellation image, we call `fetch`, or `fetchall` if the parcellation defines multiple maps. The resulting objects are a `nibabel` standard type, and can thus be directly used in many common software libraries. Here we display the maximum probability map of the Julich-Brain probabilistic maps, which comes in separate images for the left and right hemisphere.

In [ ]:
from nilearn import plotting
for img in  cytomap.fetchall():
    plotting.plot_stat_map(img)

Probabilistic atlases provide a richer continuous representation of each areal map, whicih can be accessed by explicitly selecting the continuous map type `siibra.MapType.CONTINUOUS` when requesting the map.

In [ ]:
pmaps = siibra.parcellations['julich 2.9'].get_map(siibra.spaces['mni152'],siibra.MapType.CONTINUOUS)
print(pmaps)
print(f"This map defines {len(pmaps)} different maps.")

Of course, maps and label indices of maps do represent brain regions. `siibra`'s map objects keep track of label and map indices appropriately, so we can always access the corresponding region object. This is crucial to avoid confusion between indices and regions, a common error source in data experiments that employ brain atlases.

As an example, assume we want to display the probabiliy map of v1 in the left hemisphere. Which one to fetch?

In [ ]:
# Which probability map encodes V1 in the left hemisphere?
index = pmaps.decode_region('v1 left')[0].map
from nilearn import plotting
plotting.plot_stat_map(pmaps.fetch(mapindex=index))

Vice versa, we can easily find the region of a given map index.

In [ ]:
region = pmaps.decode_label(120)
print(region)

The same works accordingly for the maximum probabily map, where the region is encoded by both the map index (here: the hemisphere) and its color, i.e. label index: 

In [ ]:
cytomap.decode_label(mapindex=0,labelindex=10)

## Reference templates and high-resolution data

Besides brain maps, each reference space also has a data representation - typically an image - which represents the space. Interesting is particularly the BigBrain, which is a microscopic model and too large to download. For such cases, `siibra` allows to fetch high-resolution data at reduced resolutions, or fetching only volumes of interest.

In [ ]:
mni152 = siibra.spaces['mni152'].get_template()
plotting.plot_img(mni152.fetch(),cmap='gray')

In [ ]:
bigbrain = siibra.spaces['bigbrain'].get_template()
plotting.view_img(bigbrain.fetch(),bg_img=None,cmap='gray')

In [ ]:
minpt = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)
bb = siibra.spaces['bigbrain']
voi = siibra.spaces['bigbrain'].get_voi(minpt,maxpt)
plotting.view_img(bigbrain.fetch(resolution_mm=0.08, voi=voi),bg_img=None,cmap='gray')

# Authenticating with EBRAINS knowledge graph

We can often fetch more information on an object from the KG. To fetch data from the KG, you need to pass an authentication token to siibra which you can get from the [authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize). To generate such tokens, you need to 

1. be registered to EBRAINS (as described [here](https://ebrains.eu/register)), and 
2. enable API access for your account (as described [here](https://kg.humanbrainproject.eu/develop.html)).

In [ ]:
token = input("Enter your token here, then press 'Enter': ")
siibra.set_ebrains_token(token)

In [ ]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS

In [ ]:
atlas.select_region('44')

In [ ]:
features = atlas.get_features(siibra.modalities.CorticalCellDistribution)